In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
import os
import json
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_style()
import pandas as pd

import jax
from jax import config
config.update("jax_enable_x64", True)
try:
    jax.devices()
except:
    config.update('jax_platforms','cpu')
    
sys.path.append(os.path.join(os.getcwd(),'repos/dysts/'))
import dysts
from dysts.datasets import *
from dysts.flows import *
from dysts.base import *
from dysts.utils import *
from dysts.analysis import *

# Darts implementation

# HS using local darts
sys.path.append(os.path.join(os.getcwd(),'repos/darts/'))
import darts
from darts.models import *
from darts import TimeSeries
import darts.models

import numpy as np
np.random.seed(0)

from copy import deepcopy

# Fitting to the loaded chaotic system

In [ ]:
input_path = "train_univariate__pts_per_period_100__periods_12.json"
#input_path = os.getcwd() + "/../dysts/dysts/data/train_univariate__pts_per_period_100__periods_12_noise.json"
#input_path = os.getcwd() + "/../dysts/benchmarks/results/results_train_univariate__pts_per_period_100__periods_12_noise.json"

dataname = os.path.splitext(os.path.basename(os.path.split(input_path)[-1]))[0]
equation_data = load_file(input_path)    


hyperparameter_path = "repos/dysts/benchmarks/hyperparameters/220812_hyperparameters_" + dataname + ".json"
#220518_hyperparameters_
with open(hyperparameter_path, "r") as file:
    all_hyperparameters = json.load(file)

In [ ]:
equation_name = "BeerRNN" 
# "Aizawa" 
# "Arneodo"
# "AnishchenkoAstakhov"
# "BelousovZhabotinsky"
# "BlinkingVortex"
# "CellularNeuralNetwork"
# "Chen"
# "Chua"
# "CircadianRhythm"
# "CoevolvingPredatorPrey"
# "Duffing"
# "Halvorsen"
# "HenonHeiles"
# "Lorenz"
# "PiecewiseCircuit"
# "Rossler"
# "Thomas"
# "Blasius"

# Error
# "GenesioTesi"

train_data = np.copy(np.array(equation_data.dataset[equation_name]["values"]))
train_data_noiseless = deepcopy(train_data)

#add noise
noise_scale = np.std(train_data[:int(5/6 * len(train_data))]) # prevent leakage
train_data += 0.2 * np.std(train_data) * np.random.normal(size=train_data.shape[0])
#train_data += 0.5 * np.std(train_data) * np.random.normal(size=train_data.shape[0])
      
split_point = int(5/6 * len(train_data))
y_train, y_val = train_data[:split_point], train_data[split_point:]
y_train_ts, y_test_ts = TimeSeries.from_dataframe(pd.DataFrame(train_data)).split_before(split_point)

    
plt.plot(train_data, lw=.1)
plt.plot(train_data_noiseless, lw=.5)
plt.plot(y_train, lw=.1)

In [ ]:
y_train;

In [ ]:
#RBF Takens

In [ ]:
# # Fitting NLSS
# model_name = "RBF_Takens" #"LSS"##"NLSS_Sampling" #"LSS" #
# print(all_hyperparameters[equation_name][model_name])
# model_RBF = getattr(darts.models, model_name)(**all_hyperparameters[equation_name][model_name])
# model_RBF.fit(y_train_ts)

In [ ]:
# # Predictiong by RBF
# tmp = model_RBF.predict(len(y_val))
# y_val_pred = {model_name: tmp}

# plt.plot(train_data, lw=.1)
# y_val_pred[model_name].plot(color="red", label="Prediction")

In [ ]:
darts.models

## LSS Takens

In [ ]:
# Fitting LSS
model_name = "LSS_Takens"
model_LSS_Takens = getattr(darts.models, model_name)(**all_hyperparameters[equation_name][model_name])
model_LSS_Takens.fit(y_train_ts)

In [ ]:
# Predictiong by LSS
tmp = model_LSS_Takens.predict(len(y_val))
y_val_pred = {model_name: tmp} 

plt.plot(train_data, lw=.1)
y_val_pred[model_name].plot(color="red", label="Prediction")

## NLSS Takens

In [ ]:
# Fitting NLSS
model_name = "NLSS_Takens" #"LSS"##"NLSS_Sampling" #"LSS" #
model_NLSS = getattr(darts.models, model_name)(**all_hyperparameters[equation_name][model_name])
model_NLSS.fit(y_train_ts)

In [ ]:
# Predictiong by NLSS
tmp = model_NLSS.predict(len(y_val))
#tmp = model_NLSS.predict_noiseless(len(y_val))
#tmp = model_NLSS.predict_sample(len(y_val))
y_val_pred = {model_name: tmp}

plt.plot(train_data, lw=.1)
y_val_pred[model_name].plot(color="red", label="Prediction")

In [ ]:
y_train[-1]

In [ ]:
# # Predictiong by NLSS
# #tmp = model_NLSS.predict(len(y_val))
# tmp = model_NLSS.predict_noiseless(len(y_val))
# y_val_pred = {model_name: tmp}

# plt.plot(train_data, lw=.1)
# y_val_pred[model_name].plot(color="red", label="Prediction")

In [ ]:
DIM = model_NLSS.ssm_emd.sm.Dk * (1 + 2*model_NLSS.ssm_emd.sm.Dz)
DIM

In [ ]:
#y_val_pred["NLSS_Takens"]

In [ ]:
# Saving data
#y_train_ts.to_pickle('y_train_ts.pkl')
y_train_ts.to_csv('y_train_ts.csv')

## ARIMA

In [ ]:
#model_name = "KalmanForecaster"
model_name = "ARIMA"
model = getattr(darts.models, model_name)(**all_hyperparameters[equation_name][model_name])
model.fit(y_train_ts)

tmp = model.predict(len(y_val))
y_val_pred = {model_name: tmp} 
#y_val_pred[model_name] 

plt.plot(train_data, lw=.1)
y_val_pred[model_name].plot(color="red", label="Prediction")

In [ ]:
model_name = "RNNModel"
model = getattr(darts.models, model_name)(**all_hyperparameters[equation_name][model_name])
model.fit(y_train_ts)

y_val_pred[model_name] = model.predict(len(y_val))

plt.plot(train_data, lw=.1)
y_val_pred[model_name].plot(color="red", label="Prediction")

In [ ]:
## TransformerModel

In [ ]:
model_name = "TransformerModel"
model = getattr(darts.models, model_name)(**all_hyperparameters[equation_name][model_name])
model.fit(y_train_ts)

y_val_pred[model_name] = model.predict(len(y_val))

plt.plot(train_data, lw=.1)
y_val_pred[model_name].plot(color="red", label="Prediction")

## NBEATSModel

In [ ]:
model_name = "NBEATSModel"
model = getattr(darts.models, model_name)(**all_hyperparameters[equation_name][model_name])
model.fit(y_train_ts)

y_val_pred[model_name] = model.predict(len(y_val))

plt.plot(train_data, lw=.1)
y_val_pred[model_name].plot(color="red", label="Prediction")

# Plot

In [ ]:
model_names = ['ARIMA','RNNModel','TransformerModel','NBEATSModel','NLSS_Takens']
#model_names = ['RNNModel','TransformerModel','NBEATSModel','NLSS_Takens']
#model_names = ['ARIMA','RNNModel','NBEATSModel','NLSS_Takens']

i = 1
for model_name in model_names:
    plt.subplot(len(model_names),1,i)
    
    
    plt.plot(train_data_noiseless, lw = .2)
    plt.plot(y_train, lw = .2)
    
    if model_name == 'NLSS_Takens':
        y_val_pred[model_name].plot(label=model_name, color='red') #
        plt.legend(loc='upper left')
    else:
        ax = y_val_pred[model_name].plot(xticks=[], color='blue', label=model_name)
        plt.legend(loc='upper left')
        plt.xlabel('')    

    plt.xlim([600, 1200]) 
    #plt.ylim([-2, 2]) 
    
    if i == 1:
        plt.title(f'{equation_name}')
        
        
    i = i + 1



# MISC

In [ ]:
y_val_pred[model_name]

In [ ]:
model_name = "NBEATSModel"
model = getattr(darts.models, model_name)(**all_hyperparameters[equation_name][model_name])
